In [7]:
from comet import download_model, load_from_checkpoint
from pathlib import Path

In [10]:
# model_path = download_model("Unbabel/wmt22-comet-da", saving_directory="models/comet_wmt22")
model_path = "models/comet_wmt22/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt"

In [ ]:
print(model_path)

C:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\translation_analysis\analysis1\models\comet_wmt22\models--Unbabel--wmt22-comet-da\snapshots¾0a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt


In [11]:
model = load_from_checkpoint(model_path)

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\translation_analysis\analysis1\models\comet_wmt22\models--Unbabel--wmt22-comet-da\snapshots\2760a223ac957f30acfb18c8aa649b01cf1d75f2\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\venv\lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [6]:
data = [
    {
        "src": "10 到 15 分钟可以送到吗",
        "mt": "Can I receive my food in 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    },
    {
        "src": "Pode ser entregue dentro de 10 a 15 minutos?",
        "mt": "Can you send it for 10 to 15 minutes?",
        "ref": "Can it be delivered between 10 to 15 minutes?"
    }
]
# Call predict method:
model_output = model.predict(data, batch_size=8, gpus=0) # quite rapid

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]


In [7]:
print(model_output)

Prediction([('scores', [0.8417138457298279, 0.7745394110679626]), ('system_score', 0.8081266283988953)])


In [ ]:
print(model_output)

Prediction([('scores', [0.8417138457298279, 0.7745394110679626]), ('system_score', 0.8081266283988953)])
